In [2]:
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np  # linear algebra
#import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA, TruncatedSVD

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections

# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

wdi = pd.read_csv("data/Selected Datav1.csv")
wdi.head(n =25)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Arab World,ARB,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,4.779008e+01,4.755839e+01,4.732760e+01,4.709162e+01,4.684421e+01,4.657710e+01,...,2.749600e+01,2.751344e+01,2.748487e+01,2.738930e+01,2.721144e+01,2.694078e+01,2.657699e+01,2.613480e+01,2.564802e+01,NaN
1,Arab World,ARB,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,6.977256e+00,6.998635e+00,7.018725e+00,7.035036e+00,7.045690e+00,7.049042e+00,...,3.475024e+00,3.470384e+00,3.463100e+00,3.450654e+00,3.430464e+00,3.402059e+00,3.365221e+00,3.322465e+00,3.276553e+00,NaN
2,Arab World,ARB,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,4.283168e-01,4.772079e+00,3.629086e+00,6.656678e+00,3.166151e+00,2.455201e+00,3.307618e+00,3.247325e+00,9.998818e-01,2.096747e+00
3,Arab World,ARB,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.363818e+04,1.412554e+04,1.451643e+04,1.542068e+04,1.582148e+04,1.615147e+04,1.650069e+04,1.691102e+04,1.710213e+04,1.755781e+04
4,Arab World,ARB,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.365192e+04,1.400315e+04,1.438016e+04,1.528592e+04,1.557512e+04,1.604909e+04,1.651546e+04,1.698540e+04,1.713191e+04,1.749890e+04
5,Arab World,ARB,Population ages 65 and above (% of total popul...,SP.POP.65UP.TO.ZS,3.507922e+00,3.539326e+00,3.564483e+00,3.583551e+00,3.596393e+00,3.603238e+00,...,4.123422e+00,4.133051e+00,4.152697e+00,4.175251e+00,4.204198e+00,4.244906e+00,4.301047e+00,4.374559e+00,4.461560e+00,4.557876e+00
6,Arab World,ARB,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,8.430860e+00,8.663154e+00,8.903441e+00,9.152526e+00,9.410965e+00,...,3.085858e+01,3.159402e+01,3.233012e+01,3.306767e+01,3.380379e+01,3.453398e+01,3.525690e+01,3.596876e+01,3.666980e+01,3.737237e+01
7,Arab World,ARB,Population growth (annual %),SP.POP.GROW,NaN,2.740584e+00,2.755287e+00,2.773671e+00,2.797625e+00,2.823683e+00,...,2.433025e+00,2.383187e+00,2.329922e+00,2.281329e+00,2.224341e+00,2.160102e+00,2.093418e+00,2.019087e+00,1.949024e+00,1.915912e+00
8,Arab World,ARB,"Population, male (% of total population)",SP.POP.TOTL.MA.ZS,5.041523e+01,5.041499e+01,5.041299e+01,5.040983e+01,5.040639e+01,5.040350e+01,...,5.161105e+01,5.167011e+01,5.170547e+01,5.172105e+01,5.172522e+01,5.172913e+01,5.173987e+01,5.175959e+01,5.178476e+01,5.180967e+01
9,Arab World,ARB,"Population, total",SP.POP.TOTL,9.219775e+07,9.472451e+07,9.733444e+07,1.000342e+08,1.028328e+08,1.057364e+08,...,3.466292e+08,3.548900e+08,3.631587e+08,3.714435e+08,3.797057e+08,3.879077e+08,3.960283e+08,4.040244e+08,4.118990e+08,4.197906e+08


In [7]:
#looking into countries data integrity
country_num = 180
country_list = []
country_remove_name = []
for i in range(country_num):
    country_list.append(wdi[wdi["Country Code"] == wdi.iloc[22*i+1,1]])
for i, cou in enumerate(country_list):
    #get null values, and remove ones with high sum by deleting country_num numbers
    if cou.isnull().sum().sum()>400:
        #print(i,":",cou.iloc[0,0],":",cou.isnull().sum().sum()) 
        country_remove_name.append(cou.iloc[0,0])
country_filtered = []
#filter out countirs
for i in country_list:
    if i.iloc[0,0] not in country_remove_name:
        country_filtered.append(i)
wdi_filtered = pd.concat(country_filtered)
country_remove_name

['Not classified',
 'American Samoa',
 'Andorra',
 'British Virgin Islands',
 'Cayman Islands',
 'Dominica',
 'Kosovo',
 'Liechtenstein',
 'Marshall Islands',
 'Monaco',
 'Nauru',
 'Northern Mariana Islands',
 'San Marino',
 'Serbia',
 'St. Kitts and Nevis',
 'St. Martin (French part)',
 'Tuvalu']

In [8]:
var_original_num = 22
var_num = [0,1,4,5,6,7,10,11,12,13,14,15,17,19,21]
var_name = []
filtered_data = []
for i in range(var_original_num):
    if i not in var_num:
        var_name.append(wdi_filtered.iloc[i,3])


#needs to deal with missing values in gdp pp,gni pp, unemployment rate. 


In [183]:
wdi_filtered.head(n = 50)

,Country Name,Country Code,Indicator Name,Indicator Code,1985,1986,1987,1988,1989,1990,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,AFG,Age dependency ratio (% of working-age populat...,SP.POP.DPND,9.926418e+01,1.000380e+02,1.003185e+02,1.004199e+02,1.008525e+02,1.021867e+02,...,1.022490e+02,1.020458e+02,1.002245e+02,9.792595e+01,9.533874e+01,9.264914e+01,8.995409e+01,8.794179e+01,8.600075e+01,8.407766e+01
1,Afghanistan,AFG,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,4.973500e+01,4.958600e+01,4.941800e+01,4.923600e+01,4.904800e+01,4.888000e+01,...,4.090300e+01,3.982900e+01,3.875000e+01,3.769000e+01,3.667000e+01,3.570600e+01,3.480900e+01,3.398100e+01,3.321100e+01,NaN
4,Afghanistan,AFG,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,7.458000e+00,7.460000e+00,7.461000e+00,7.461000e+00,7.461000e+00,7.466000e+00,...,6.180000e+00,5.977000e+00,5.770000e+00,5.562000e+00,5.359000e+00,5.163000e+00,4.976000e+00,4.800000e+00,4.633000e+00,NaN
5,Afghanistan,AFG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,2.139053e+01,1.436244e+01,4.263548e-01,1.275229e+01,5.600745e+00,2.724543e+00,1.451315e+00,2.260314e+00,2.665292e+00,1.030660e+00
6,Afghanistan,AFG,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.454663e+03,1.637378e+03,1.626765e+03,1.806764e+03,1.874766e+03,1.897526e+03,1.886693e+03,1.896993e+03,1.934637e+03,1.951559e+03
7,Afghanistan,AFG,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.450000e+03,1.640000e+03,1.620000e+03,1.810000e+03,1.880000e+03,1.900000e+03,1.900000e+03,1.910000e+03,1.940000e+03,1.960000e+03
10,Afghanistan,AFG,Population ages 65 and above (% of total popul...,SP.POP.65UP.TO.ZS,2.239398e+00,2.271446e+00,2.294741e+00,2.302779e+00,2.285279e+00,2.232542e+00,...,2.305590e+00,2.328132e+00,2.354462e+00,2.386294e+00,2.420207e+00,2.451575e+00,2.478662e+00,2.519923e+00,2.554251e+00,2.584927e+00
11,Afghanistan,AFG,Population density (people per sq. km of land ...,EN.POP.DNST,1.828602e+01,1.797656e+01,1.777492e+01,1.779555e+01,1.817982e+01,1.901220e+01,...,4.349296e+01,4.470408e+01,4.613150e+01,4.773056e+01,4.942804e+01,5.111478e+01,5.271207e+01,5.419711e+01,5.559599e+01,5.693776e+01
12,Afghanistan,AFG,Population growth (annual %),SP.POP.GROW,-2.204368e+00,-1.706772e+00,-1.128041e+00,1.160166e-01,2.136355e+00,4.476897e+00,...,2.397021e+00,2.746576e+00,3.143126e+00,3.407587e+00,3.494589e+00,3.355582e+00,3.077084e+00,2.778317e+00,2.548347e+00,2.384761e+00
13,Afghanistan,AFG,"Population, male (% of total population)",SP.POP.TOTL.MA.ZS,5.102852e+01,5.102616e+01,5.099864e+01,5.097835e+01,5.101028e+01,5.111552e+01,...,5.122083e+01,5.117524e+01,5.118422e+01,5.123289e+01,5.129993e+01,5.135868e+01,5.139295e+01,5.140033e+01,5.138838e+01,5.136415e+01


In [31]:
sun = pd.read_csv('Annual_sunshine_hours.csv')
sun.rename(columns = {"Annual NCDC Computed Value":'Hours_per_year'},inplace = True)
sun = sun[sun.Hours_per_year != 0.0]
sun = sun.groupby('Country or Territory').Hours_per_year.mean()
sun

Country or Territory
1                                                             NaN
ARGENTINA                                             1984.011765
ARMENIA                                               2457.500000
AUSTRALIA                                             2746.770000
AUSTRALIA (ANTARCTIC STATIONS)                        1644.150000
BAHAMAS                                               2878.400000
BELGIUM                                               1489.600000
BELIZE                                                1915.900000
BRAZIL                                                2270.202475
BULGARIA                                              2182.857143
COLOMBIA                                              1981.693750
COSTA RICA                                            2257.800000
CZECH REPUBLIC                                        1584.025000
FINLAND                                               1664.440000
FRANCE                                                2